1 - Configura o spark

In [1]:
# Para que o Jupyter consiga carregar o Spark corretamente no notebook
import findspark
findspark.init('/usr/local/spark')

# Para que os executors tenham mais memória e não falhem por falta de recursos
import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--executor-memory 1G pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages=org.apache.hadoop:hadoop-aws:2.7.3 --executor-memory 1G pyspark-shell'

# A partir daqui é código Spark que normalmente é executado com um comando similar ao comando abaixo:
# spark-submit --executor-memory 1G nome_do_script.py
from pyspark import SparkConf, SparkContext

# A linha abaixo está comentada porque essa é a forma de executar Spark em uma instalação local usando todos os cores
#conf = SparkConf().setMaster("local[*]").setAppName("NomeDoApp")

conf = SparkConf()
sc = SparkContext(conf=conf)

2 - Importar libraries

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

import collections

3 - Cria uma Spark Session

In [3]:
spark = SparkSession.builder.master("local").appName("SparkSQL").getOrCreate()

4 - Carrega o data set dos clientes, dá um parser em cada linha do data set, cria uma dataframe e cria uma view temporária

In [4]:
def parse_customers(line):
    fields = line.split(',')
    return Row(timestamp=fields[0],
               user_id=fields[1],
               lat=fields[2],
               lng=fields[3])


data = spark.sparkContext.textFile("./datasets/problema-06-sample-01.csv")
customers = data.map(parse_customers)
customers_df = spark.createDataFrame(customers).cache()
customers_df.createOrReplaceTempView("customers")

5 - Testa se o passo anterior(4) funcionou

In [ ]:
customers_df.groupBy("user_id").count().orderBy("count", ascending=False).show(5)

6 - Carrega o data set das lojas, dá um parser em cada linha do data set, cria uma dataframe e cria uma view temporária

In [5]:
def parse_stores(line):
    fields = line.split(',')
    return Row(lat=fields[0],
               lng=fields[1],
               category=fields[2])


data = spark.sparkContext.textFile("./datasets/problema-06_estabelecimentos.csv")
stores = data.map(parse_stores)
stores_df = spark.createDataFrame(stores).cache()
stores_df.createOrReplaceTempView("stores")

7 - Testa se o passo anterior(6) funcionou

In [ ]:
stores_df.groupBy("category").count().orderBy("count", ascending=False).show(5)

8 - Junta as tabelas customers e stores de acordo com as colunas 'lng' e 'lat' seguindo o critério de igualdade

In [6]:
customers_stores_df = customers_df.join(stores_df, ["lng", "lat"], 'inner')

9 - Testa se o passo anterior(8) funcionou

In [ ]:
customers_stores_df.show(5)

10 - Importa a função dayofweek e cria uma nova coluna 'weekday' com a informação do dia da semana extraida da coluna 'timestamp'

In [7]:
from pyspark.sql.functions import dayofweek

customers_stores_with_weekday_df = customers_stores_df.withColumn("weekday", dayofweek('timestamp'))

11 - Testa se o passo anterior(9) funcionou

In [ ]:
customers_stores_with_weekday_df.show(5)

In [8]:
# from pyspark.sql.functions import desc

counter_customers_visits_df = customers_stores_with_weekday_df.groupby('user_id').count().orderBy("count", ascending=False)

In [ ]:
counter_customers_visits_df.show(5)

In [9]:
top_customers_id_df = counter_customers_visits_df.limit(10).select("user_id")

In [ ]:
top_customers_id_df.show(5)
print(top_customers_id_df.count())

In [10]:
customers_stores_with_weekday_df.filter(customers_stores_with_weekday_df.user_id == top_customers_id_df.user_id)
print(customers_stores_with_weekday_df.count())

66367


In [15]:
grouped_results = customers_stores_with_weekday_df['user_id', 'category', 'weekday'].groupby(['user_id', 'category', 'weekday']).count().orderBy("count", ascending=False)

In [16]:
grouped_results.show(5)

+--------------------+-------------+-------+-----+
|             user_id|     category|weekday|count|
+--------------------+-------------+-------+-----+
|d31143fc-4a99-443...| 'real state'|      6|   61|
|d31143fc-4a99-443...| 'real state'|      7|   58|
|4a76753a-e399-4a7...|'electronics'|      6|   57|
|d31143fc-4a99-443...| 'real state'|      1|   57|
|e40aff9a-380e-479...|'restaurants'|      3|   56|
+--------------------+-------------+-------+-----+
only showing top 5 rows



In [ ]:
spark.stop()